In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

e:\New folder (2)\Desktop\Projects\MLOps-end-to-end-Project\atlas\Lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
260,There are exactly 2 good things to be said abo...,negative
131,The only thing worse than surfers without any ...,negative
438,Like in a circle the movie leads back to its p...,positive
940,For me personally this film goes down in my to...,positive
24,We're a long way from LAURA. Once again Otto P...,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
260,exactly good thing said fantasy both mentioned...,negative
131,thing worse surfer without wave film surfer wi...,negative
438,like circle movie lead back point departure im...,positive
940,personally film go top four time exception jam...,positive
24,long way laura otto preminger directs dana and...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
negative    253
positive    247
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
260,exactly good thing said fantasy both mentioned...,0
131,thing worse surfer without wave film surfer wi...,0
438,like circle movie lead back point departure im...,1
940,personally film go top four time exception jam...,1
24,long way laura otto preminger directs dana and...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Gaurav9693089415/MLOps-end-to-end-Project.mlflow')
dagshub.init(repo_owner='Gaurav9693089415', repo_name='MLOps-end-to-end-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic_Regression_Baseline")


Accessing as Gaurav9693089415

Initialized MLflow to track repo "Gaurav9693089415/MLOps-end-to-end-Project"

Repository Gaurav9693089415/MLOps-end-to-end-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/9e8ed88df9074ff490cf4d80a255dc95', creation_time=1759850177020, experiment_id='0', last_update_time=1759850177020, lifecycle_stage='active', name='Logistic_Regression_Baseline', tags={}>

In [15]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        #Save and log the notebook
        #notebook_path = "exp1_baseline_model.ipynb"
        #logging.info("Executing Jupyter Notebook. This may take a while...")
        #os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        #mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-10-07 22:01:27,531 - INFO - Starting MLflow run...
2025-10-07 22:01:28,785 - INFO - Logging preprocessing parameters...
2025-10-07 22:01:30,164 - INFO - Initializing Logistic Regression model...
2025-10-07 22:01:30,169 - INFO - Fitting the model...
2025-10-07 22:01:30,178 - INFO - Model training complete.
2025-10-07 22:01:30,178 - INFO - Logging model parameters...
2025-10-07 22:01:30,601 - INFO - Making predictions...
2025-10-07 22:01:30,602 - INFO - Calculating evaluation metrics...
2025-10-07 22:01:30,609 - INFO - Logging evaluation metrics...
2025-10-07 22:01:32,411 - INFO - Saving and logging the model...
e:\New folder (2)\Desktop\Projects\MLOps-end-to-end-Project\atlas\Lib\site-packages\_distutils_hack\__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptool